In [40]:
import tensorflow as tf
import pandas as pd
import tensorflow_hub as hub
import numpy as np
import cv2
import joblib

In [22]:
data_path =("/content/drive/MyDrive/copy paste")

In [23]:
IMAGE_SHAPE = (224, 224) # (height, width) in no. of pixels

In [24]:
TRAINING_DATA_DIR = str(data_path)
datagen_kwargs = dict(rescale=1./255, validation_split=.25)

In [25]:
valid_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
valid_generator = valid_datagen.flow_from_directory(TRAINING_DATA_DIR,subset="validation", shuffle=True,target_size=IMAGE_SHAPE)
train_datagen = tf.keras.preprocessing.image.ImageDataGenerator(**datagen_kwargs)
train_generator = train_datagen.flow_from_directory(TRAINING_DATA_DIR,subset="training",shuffle=True,target_size=IMAGE_SHAPE)

Found 350 images belonging to 2 classes.
Found 1050 images belonging to 2 classes.


In [26]:
print (train_generator.class_indices)
train_generator.image_shape
valid_generator.image_shape

{'original': 0, 'tampered': 1}


(224, 224, 3)

In [27]:
from keras.models import Sequential
from keras.layers import Dense, Dropout, Conv2D, MaxPool2D, Flatten
from keras.utils import np_utils

In [34]:
# building a linear stack of layers with the sequential model
model = Sequential()
# convolutional layer
model.add(Conv2D(25, kernel_size=(3,3), strides=(1,1), padding='valid', activation='relu', input_shape=(224,224,3)))
model.add(MaxPool2D(pool_size=(1,1)))
# flatten output of conv
model.add(Flatten())
# hidden layer
model.add(Dense(100, activation='relu'))
# output layer
model.add(Dense(2, activation='softmax'))


In [35]:
model.summary()

Model: "sequential_4"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_4 (Conv2D)            (None, 222, 222, 25)      700       
_________________________________________________________________
max_pooling2d_4 (MaxPooling2 (None, 222, 222, 25)      0         
_________________________________________________________________
flatten_2 (Flatten)          (None, 1232100)           0         
_________________________________________________________________
dense_4 (Dense)              (None, 100)               123210100 
_________________________________________________________________
dense_5 (Dense)              (None, 2)                 202       
Total params: 123,211,002
Trainable params: 123,211,002
Non-trainable params: 0
_________________________________________________________________


In [36]:
optimizer = tf.keras.optimizers.Adam(lr=1e-3)
model.compile(optimizer=optimizer,loss='categorical_crossentropy', metrics=['acc'])

In [39]:
steps_per_epoch = np.ceil(train_generator.samples/train_generator.batch_size)
val_steps_per_epoch = np.ceil(valid_generator.samples/valid_generator.batch_size)
hist = model.fit(train_generator, epochs=30,verbose=1,steps_per_epoch=3,validation_data=valid_generator,validation_steps=val_steps_per_epoch).history

Epoch 1/30
3/3 [==============================] - 20s 5s/step - loss: 3.9386 - acc: 0.6146 - val_loss: 1.8296 - val_acc: 0.4429
Epoch 2/30
3/3 [==============================] - 18s 7s/step - loss: 1.3976 - acc: 0.7188 - val_loss: 2.2950 - val_acc: 0.4914
Epoch 3/30
3/3 [==============================] - 18s 6s/step - loss: 1.2909 - acc: 0.6562 - val_loss: 4.1647 - val_acc: 0.5057
Epoch 4/30
3/3 [==============================] - 15s 5s/step - loss: 1.2105 - acc: 0.7083 - val_loss: 4.4685 - val_acc: 0.5057
Epoch 5/30
3/3 [==============================] - 16s 5s/step - loss: 0.8965 - acc: 0.7812 - val_loss: 1.2323 - val_acc: 0.3829
Epoch 6/30
3/3 [==============================] - 15s 6s/step - loss: 0.8653 - acc: 0.6875 - val_loss: 1.4222 - val_acc: 0.4600
Epoch 7/30
3/3 [==============================] - 13s 5s/step - loss: 0.3317 - acc: 0.8750 - val_loss: 3.2642 - val_acc: 0.5229
Epoch 8/30
3/3 [==============================] - 13s 5s/step - loss: 0.3099 - acc: 0.8222 - val_loss: 1